In [12]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [13]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-01-05 03:52:45--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.18MB/s    in 0.8s    

2022-01-05 03:52:47 (1.18 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Crime").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [15]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/texas_df_agencies.csv"
spark.sparkContext.addFile(url)
texas_df_agencies_df = spark.read.csv(SparkFiles.get("texas_df_agencies.csv"), sep=",", header=True, inferSchema=True)

In [16]:
# Show DataFrame
texas_df_agencies_df.show()

+---------+---------------+----------------+----------+-------------------+------------------+--------------------+
|AGENCY_ID|PUB_AGENCY_NAME|AGENCY_TYPE_NAME|POPULATION|POPULATION_GROUP_ID|SUBURBAN_AREA_FLAG|         COUNTY_NAME|
+---------+---------------+----------------+----------+-------------------+------------------+--------------------+
|    18753|      Palestine|            City|     17625|                  9|                 N|            ANDERSON|
|    18754|      Frankston|            City|      1164|                 11|                 N| HENDERSON; ANDERSON|
|    18757|        Andrews|            City|     14487|                  9|                 N|             ANDREWS|
|    18759|         Diboll|            City|      5182|                 10|                 N|            ANGELINA|
|    18760|     Huntington|            City|      2110|                 11|                 N|            ANGELINA|
|    18761|         Lufkin|            City|     35007|                 

In [17]:
url ="https://datacrime-bucket.s3.us-east-2.amazonaws.com/texas_df_crimes.csv"
spark.sparkContext.addFile(url)
texas_df_crimes_df = spark.read.csv(SparkFiles.get("texas_df_crimes.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
texas_df_crimes_df.show()

+----------+-----------+---------------+-----------+---------+---------+-------------+-------------+-----------------+-----------+----------------+-----------+-----------+--------------+------------------+-----------+------------+-----------+----------------+---------+--------------+--------------+------------+-------------+------------+-----------------+--------------+-------------+---------------------+
|OFFENSE_ID|INCIDENT_ID|OFFENSE_TYPE_ID|LOCATION_ID|DATA_YEAR|AGENCY_ID|INCIDENT_DATE|INCIDENT_HOUR|CLEARED_EXCEPT_ID|OFFENDER_ID|OFFENDER_SEQ_NUM|ARRESTEE_ID|ARREST_DATE|ARREST_TYPE_ID|MULTIPLE_INDICATOR|ARR_AGE_NUM|ARR_SEX_CODE|ARR_RACE_ID|ARR_ETHNICITY_ID|VICTIM_ID|VICTIM_SEQ_NUM|VICTIM_TYPE_ID|VICT_AGE_NUM|VICT_SEX_CODE|VICT_RACE_ID|VICT_ETHNICITY_ID|  OFFENSE_NAME|CRIME_AGAINST|OFFENSE_CATEGORY_NAME|
+----------+-----------+---------------+-----------+---------+---------+-------------+-------------+-----------------+-----------+----------------+-----------+-----------+-----------

In [18]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')


Enter database password··········


In [19]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-crime.czbxze2tryse.us-east-2.rds.amazonaws.com:5432/database-crime"
config = {"user":"postgres",
          "password": "bootcamp10",
          "driver":"org.postgresql.Driver"}

In [20]:
# Write DataFrame to texas_df_agencies_df table in RDS
texas_df_agencies_df.write.jdbc(url=jdbc_url, table='texas_df_agencies', mode=mode, properties=config)

In [11]:
# Write DataFrame to texas_df_crimes_df table in RDS
texas_df_crimes_df.write.jdbc(url=jdbc_url, table='texas_df_crimes', mode=mode, properties=config)